In [1]:
import os
import argparse
import logging
logging.basicConfig(level=logging.DEBUG)
from common import find_mxnet, data, fit
from common.util import download_file
from tf_iterators import *
import mxnet as mx
import numpy as np
import numpy 
from mxnet import nd
#from mxnet.module.module_tf import *

In [2]:
def inat_iterator(data_dir,subset,batch_size):
    train = mx.io.ImageRecordIter(
            path_imglist=os.path.join(data_dir, "inat_"+subset+"_train.lst"),
            path_imgrec         = os.path.join(data_dir, "inat_"+subset+"_train.rec"),
            label_width         = 2,
            data_name           = 'data',
            label_name          = 'softmax1_label',
            data_shape          = (3, 224, 224),
            batch_size          = batch_size,
            pad                 = 0,
            fill_value          = 127,  # only used when pad is valid
            rand_crop           = True,
            max_random_scale    = 1.0,  # 480 with imagnet, 32 with cifar10 448 with birds 0.93
            min_random_scale    = 0.53,  # 256.0/480.0
            max_aspect_ratio    =  0.25,
            random_h            = 36,  # 0.4*90
            random_s            = 50,  # 0.4*127
            random_l            = 50,  # 0.4*127
            #max_rotate_angle    = 10,
            #max_shear_ratio     = 0.1, #
            rand_mirror         = True,
            shuffle             = True)
            #num_parts           = kv.num_workers,
            #part_index          = kv.rank)
    val = mx.io.ImageRecordIter(
            path_imgrec         = os.path.join(data_dir, "inat_"+subset+"_val.rec"),
            path_imglist=os.path.join(data_dir, "inat_"+subset+"_val.lst"),
            label_width         = 2,
            data_name           = 'data',
            label_name          = 'softmax1_label',
            batch_size          = batch_size,
            max_random_scale    = 0.53,  # 480 with imagnet, 32 with cifar10
            min_random_scale    = 0.53,  # 256.0/480.
            data_shape          = (3, 224, 224),
            rand_crop           = False,
            rand_mirror         = False)
            #num_parts           = kv.num_workers,
            #part_index          = kv.rank)
    return train, val


class inat_sub_iterator(mx.io.DataIter):
    '''multi label ilab iterator'''

    def __init__(self, data_iter,subset):
        super(inat_sub_iterator, self).__init__()
        self.data_iter = data_iter
        self.batch_size = self.data_iter.batch_size
        self.labelIdx=subset

    @property
    def provide_data(self):
        return self.data_iter.provide_data

    @property
    def provide_label(self):
        provide_label = self.data_iter.provide_label
        label_names=[]
        batch_size=[]
        for i in range(len(self.labelIdx)):
            label_names.append('softmax%d_label'%(i+1))
            batch_size.append((self.batch_size,))
        return zip(label_names,batch_size)  
  
    def hard_reset(self):
        self.data_iter.hard_reset()

    def reset(self):
        self.data_iter.reset()

    def next(self):
        batch = self.data_iter.next()
        labelnp=[]
        for lab in batch.label[0].T.asnumpy():
            #print lab.shape
            labelnp.append(mx.nd.array(lab))        
        all_label = [labelnp[i] for i in self.labelIdx]
        return mx.io.DataBatch(data=batch.data, label=all_label, \
                pad=batch.pad, index=batch.index)

In [3]:
class Cross_Entropy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self):
        super(Cross_Entropy, self).__init__('cross-entropy')
    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)
        label = labels[0].asnumpy()
        pred = preds[0].asnumpy()
        for i in range(label.shape[0]):
            prob = pred[i,numpy.int64(label[i])]
            if len(labels) == 1:
                self.sum_metric += (-numpy.log(prob)).sum()
        self.num_inst += label.shape[0]

In [4]:
image_shape = '3,224,224'
batch_size = 128
results_prefix='/home/ubuntu/results/'

#results_prefix='/efs/users/furlat/v1_results/'http://127.0.0.1:8888/notebooks/example/image-classification/v2_single_task_template.ipynb#

In [5]:
subset='Mammalia'
nstream=1
init_1='imagenet'
init_2='imagenet'
freeze=0
num_layers=101


In [6]:
print subset
data_dir='/home/ubuntu/data/inat'
train_inat, val_inat = inat_iterator(data_dir,subset,batch_size)
train_inat=inat_sub_iterator(train_inat,[0])
val_inat=inat_sub_iterator(val_inat,[0])

num_classes=5089

print "mona"


Mammalia
mona


In [7]:


    
#elif dataset =='sketchy':
#    data_dir_sketchy='/efs/datasets/users/furlat/sketchy_database'
#    train_sketches, val_sketches = sketchy_iterator(data_dir_sketchy,batch_size)
#    num_classes=125
label_names = [train_inat.provide_label[0][0]]
    
from importlib import import_module
net = import_module('symbols.resnet_factory')

if nstream == 1:
    print 'plain resnet'

    ctx=[mx.gpu(4),mx.gpu(5),mx.gpu(6),mx.gpu(7)]    
    #ctx=[mx.gpu(0+(3*gpu_block)),mx.gpu(1+(3*gpu_block)),mx.gpu(2+(3*gpu_block))]
    #print 'gpu bloc%2d : using gpu %2d to %2d' %(gpu_block,0+(3*gpu_block),2+(3*gpu_block))

    arch='resnet'    
    sym,data,labels=net.get_symbol(num_classes=num_classes,active=[1], gate_prefix=None,rescale_grad=1, num_layers=num_layers,gated=False, image_shape=image_shape)
elif nstream==2:
    print 'multi branch resnet - Good luck daddy ;)'
    #with batch size 256  5*9686M +1*10550 ~6 full k80 Speed: 52.49 samples/sec
    #if gpu_block == 1:
        #print 'gpu block1: using gpu 0 to 6'
        
        #ctx=[mx.gpu(0),mx.gpu(1),mx.gpu(2),mx.gpu(3),mx.gpu(4),mx.gpu(5),mx.gpu(6)]
#     elif gpu_block == 2:
#         print 'gpu block2: using gpu 6 to 11'
#         ctx=[mx.gpu(7),mx.gpu(8),mx.gpu(9),mx.gpu(10),mx.gpu(11),mx.gpu(12),mx.gpu(13)]
#     elif gpu_block > 3:
#         print 'gpu bloc3 is too small for this taks enjoy the error soon'
    ctx=[mx.gpu(0)]     
    arch='mpath'
    sym,data,labels=net.get_symbol_2branch(num_classes=250,active=1, gate_prefix=None,rescale_grad=1, num_layers=50,gated=True, image_shape=image_shape,coupled=True)
    

plain resnet
mio shape [(1L, 2048L, 7L, 7L)]
mio shape [(1L, 2048L, 1L, 1L)]


In [8]:
#mx.viz.plot_network(symbol=sym)

imagenet_weights= '/home/ubuntu/models/imagenet_r50-lr05'
tornado_weights='/home/ubuntu/models/resnet-101'
#model_prefix= 'tryhard-resnet'
prefix = tornado_weights
#prefix = model_prefix
epoch=0
save_dict = nd.load('%s-%04d.params' % (prefix, epoch))
arg_params_imag = {}
aux_params_imag = {}
ext_check=['sc','fc1','data']
imagenet_par=[]
exact_check=['bn1_beta','bn1_gamma']
for k, v in save_dict.items():
    tp, name = k.split(':', 1)

    if tp == 'arg':
        arg_params_imag[name] = v
        
        
        #print name
        if not any(ext in name for ext in ext_check):
            if not any(ext == name for ext in exact_check):
                imagenet_par.append(name)
                if init_2=='imagenet':
                    arg_params_imag['_a_'+name] = v


    if tp == 'aux':
        aux_params_imag[name] = v
        if init_2=='imagenet':
            aux_params_imag['_a_'+name] = v
del arg_params_imag['fc1_bias']
del arg_params_imag['fc1_weight']



#arg_params_imag.list_arguments


#_a_bn_data_beta

symlist=sym.list_arguments()
gatelist=[s for s in symlist if 'gate' in s]

In [9]:
#imagenet_par

In [10]:


if init_1=='imagenet':
    arg_params=arg_params_imag
    aux_params=aux_params_imag
    print 'initizalizing left stream from imagenet'
    if init_2=='imagenet':
        print 'initizalizing right stream from imagenet'
else:
    arg_params=None
    aux_params=None
    print 'initizalizing left stream from random'
    
    
if nstream == 2:
    model_prefix = arch+'-'+str(num_layers)+'-fix'+str(freeze)+'-'+logistic_init+'-'+init_1+'-'+init_2+'-'+subset
    if init_2=='rand':
        print 'initizalizing right stream from random'
    print model_prefix
    if freeze==1:
        
        fixed=imagenet_par
    else:
        fixed=None
else:
    #only the parameters from the original 1stream imagenet network
    model_prefix =  arch+'-'+str(num_layers)+'-fix'+str(freeze)+'-'+init_1+'-'+subset
    print model_prefix
    if freeze==1:
        
        fixed=imagenet_par
    else:
        fixed=None

mod = mx.mod.Module(sym, label_names=label_names,fixed_param_names=fixed,context=ctx)
#
checkpoint_path=results_prefix+model_prefix
#checkpoint = mx.callback.module_checkpoint(mod,model_prefix)
mod.bind(data_shapes=train_inat.provide_data, label_shapes=train_inat.provide_label)
mod.init_params(initializer=mx.initializer.Uniform(0.01), arg_params=arg_params, aux_params=aux_params,
                    allow_missing=True, force_init=False)

checkpoint = mx.callback.module_checkpoint(mod,checkpoint_path,period=1)

# for gate in gatelist:
#     print mod.get_params()[0][gate].asnumpy(), gate

# In[7]:




initizalizing left stream from imagenet
initizalizing right stream from imagenet
resnet-101-fix0-imagenet-Mammalia


In [11]:
begin_epoch=0
if begin_epoch>0:
    sym, arg_params, aux_params =mx.model.load_checkpoint(checkpoint_path, begin_epoch)

    mod.set_params(arg_params, aux_params)
    
# if logistic_init:
#     logistic_path='/home/ubuntu/results/resnet-fix1-imagenet-birds'
#     _, arg_params, aux_params =mx.model.load_checkpoint(logistic_path, 90)

#     mod.set_params(arg_params, aux_params)    

In [ ]:
mod.fit(train_inat,
         eval_data=val_inat,
         eval_metric=[Cross_Entropy()],
         #eval_metric=[mx.metric.Accuracy()],

         batch_end_callback = [mx.callback.log_train_metric(1),mx.callback.Speedometer(batch_size,100)],
         epoch_end_callback=checkpoint,
         allow_missing=False,
         begin_epoch=begin_epoch,
         log_prefix = model_prefix,
         optimizer_params={'learning_rate':0.1, 'momentum': 0.9,'wd':0.0004 },
         num_epoch=90)

INFO:root:Iter[0] Batch[0] Train-cross-entropy=6.129931
INFO:root:Iter[0] Batch[1] Train-cross-entropy=6.122728
INFO:root:Iter[0] Batch[2] Train-cross-entropy=5.835947
INFO:root:Iter[0] Batch[3] Train-cross-entropy=5.726488
INFO:root:Iter[0] Batch[4] Train-cross-entropy=5.687597
INFO:root:Iter[0] Batch[5] Train-cross-entropy=5.604155
INFO:root:Iter[0] Batch[6] Train-cross-entropy=5.549716
INFO:root:Iter[0] Batch[7] Train-cross-entropy=5.485816
INFO:root:Iter[0] Batch[8] Train-cross-entropy=5.393551
INFO:root:Iter[0] Batch[9] Train-cross-entropy=5.342035
INFO:root:Iter[0] Batch[10] Train-cross-entropy=5.292745
INFO:root:Iter[0] Batch[11] Train-cross-entropy=5.255434
INFO:root:Iter[0] Batch[12] Train-cross-entropy=5.235859
INFO:root:Iter[0] Batch[13] Train-cross-entropy=5.166961
INFO:root:Iter[0] Batch[14] Train-cross-entropy=5.125907
INFO:root:Iter[0] Batch[15] Train-cross-entropy=5.087789
INFO:root:Iter[0] Batch[16] Train-cross-entropy=5.055333
INFO:root:Iter[0] Batch[17] Train-cross-e

INFO:root:Iter[0] Batch[140] Train-cross-entropy=2.844321
INFO:root:Iter[0] Batch[141] Train-cross-entropy=2.840894
INFO:root:Iter[0] Batch[142] Train-cross-entropy=2.834848
INFO:root:Iter[0] Batch[143] Train-cross-entropy=2.837141
INFO:root:Iter[0] Batch[144] Train-cross-entropy=2.838749
INFO:root:Iter[0] Batch[145] Train-cross-entropy=2.841924
INFO:root:Iter[0] Batch[146] Train-cross-entropy=2.845582
INFO:root:Iter[0] Batch[147] Train-cross-entropy=2.847061
INFO:root:Iter[0] Batch[148] Train-cross-entropy=2.838773
INFO:root:Iter[0] Batch[149] Train-cross-entropy=2.836404
INFO:root:Iter[0] Batch[150] Train-cross-entropy=2.836910
INFO:root:Iter[0] Batch[151] Train-cross-entropy=2.832158
INFO:root:Iter[0] Batch[152] Train-cross-entropy=2.830943
INFO:root:Iter[0] Batch[153] Train-cross-entropy=2.832574
INFO:root:Iter[0] Batch[154] Train-cross-entropy=2.832156
INFO:root:Iter[0] Batch[155] Train-cross-entropy=2.828170
INFO:root:Iter[0] Batch[156] Train-cross-entropy=2.826957
INFO:root:Iter

saving logfiles0  at resnet-101-fix0-imagenet-Mammalia-metric-log-0.txt


INFO:root:Iter[1] Batch[0] Train-cross-entropy=2.581246
INFO:root:Iter[1] Batch[1] Train-cross-entropy=2.419458
INFO:root:Iter[1] Batch[2] Train-cross-entropy=2.427020
INFO:root:Iter[1] Batch[3] Train-cross-entropy=2.486041
INFO:root:Iter[1] Batch[4] Train-cross-entropy=2.471884
INFO:root:Iter[1] Batch[5] Train-cross-entropy=2.373079
INFO:root:Iter[1] Batch[6] Train-cross-entropy=2.405879
INFO:root:Iter[1] Batch[7] Train-cross-entropy=2.423462
INFO:root:Iter[1] Batch[8] Train-cross-entropy=2.436235
INFO:root:Iter[1] Batch[9] Train-cross-entropy=2.475162
INFO:root:Iter[1] Batch[10] Train-cross-entropy=2.509613
INFO:root:Iter[1] Batch[11] Train-cross-entropy=2.510352
INFO:root:Iter[1] Batch[12] Train-cross-entropy=2.503514
INFO:root:Iter[1] Batch[13] Train-cross-entropy=2.491845
INFO:root:Iter[1] Batch[14] Train-cross-entropy=2.498490
INFO:root:Iter[1] Batch[15] Train-cross-entropy=2.481476
INFO:root:Iter[1] Batch[16] Train-cross-entropy=2.475275
INFO:root:Iter[1] Batch[17] Train-cross-e

INFO:root:Iter[1] Batch[142] Train-cross-entropy=2.282016
INFO:root:Iter[1] Batch[143] Train-cross-entropy=2.275891
INFO:root:Iter[1] Batch[144] Train-cross-entropy=2.280984
INFO:root:Iter[1] Batch[145] Train-cross-entropy=2.282819
INFO:root:Iter[1] Batch[146] Train-cross-entropy=2.280719
INFO:root:Iter[1] Batch[147] Train-cross-entropy=2.284540
INFO:root:Iter[1] Batch[148] Train-cross-entropy=2.284411
INFO:root:Iter[1] Batch[149] Train-cross-entropy=2.281687
INFO:root:Iter[1] Batch[150] Train-cross-entropy=2.282416
INFO:root:Iter[1] Batch[151] Train-cross-entropy=2.276205
INFO:root:Iter[1] Batch[152] Train-cross-entropy=2.277252
INFO:root:Iter[1] Batch[153] Train-cross-entropy=2.281344
INFO:root:Iter[1] Batch[154] Train-cross-entropy=2.281889
INFO:root:Iter[1] Batch[155] Train-cross-entropy=2.283003
INFO:root:Iter[1] Batch[156] Train-cross-entropy=2.287953
INFO:root:Iter[1] Batch[157] Train-cross-entropy=2.286711
INFO:root:Iter[1] Batch[158] Train-cross-entropy=2.288630
INFO:root:Iter

...

INFO:root:Iter[4] Batch[0] Train-cross-entropy=1.668364
INFO:root:Iter[4] Batch[1] Train-cross-entropy=1.755910
INFO:root:Iter[4] Batch[2] Train-cross-entropy=1.740058
INFO:root:Iter[4] Batch[3] Train-cross-entropy=1.809355
INFO:root:Iter[4] Batch[4] Train-cross-entropy=1.827996
INFO:root:Iter[4] Batch[5] Train-cross-entropy=1.820596
INFO:root:Iter[4] Batch[6] Train-cross-entropy=1.816377
INFO:root:Iter[4] Batch[7] Train-cross-entropy=1.822060
INFO:root:Iter[4] Batch[8] Train-cross-entropy=1.823628
INFO:root:Iter[4] Batch[9] Train-cross-entropy=1.843153
INFO:root:Iter[4] Batch[10] Train-cross-entropy=1.883887
INFO:root:Iter[4] Batch[11] Train-cross-entropy=1.894422
INFO:root:Iter[4] Batch[12] Train-cross-entropy=1.903012
INFO:root:Iter[4] Batch[13] Train-cross-entropy=1.884740
INFO:root:Iter[4] Batch[14] Train-cross-entropy=1.888837
INFO:root:Iter[4] Batch[15] Train-cross-entropy=1.899505
INFO:root:Iter[4] Batch[16] Train-cross-entropy=1.894538
INFO:root:Iter[4] Batch[17] Train-cross-e

INFO:root:Iter[4] Batch[142] Train-cross-entropy=1.826532
INFO:root:Iter[4] Batch[143] Train-cross-entropy=1.823281
INFO:root:Iter[4] Batch[144] Train-cross-entropy=1.819225
INFO:root:Iter[4] Batch[145] Train-cross-entropy=1.821637
INFO:root:Iter[4] Batch[146] Train-cross-entropy=1.824061
INFO:root:Iter[4] Batch[147] Train-cross-entropy=1.821964
INFO:root:Iter[4] Batch[148] Train-cross-entropy=1.824891
INFO:root:Iter[4] Batch[149] Train-cross-entropy=1.822901
INFO:root:Iter[4] Batch[150] Train-cross-entropy=1.821205
INFO:root:Iter[4] Batch[151] Train-cross-entropy=1.817033
INFO:root:Iter[4] Batch[152] Train-cross-entropy=1.822771
INFO:root:Iter[4] Batch[153] Train-cross-entropy=1.820946
INFO:root:Iter[4] Batch[154] Train-cross-entropy=1.823020
INFO:root:Iter[4] Batch[155] Train-cross-entropy=1.829906
INFO:root:Iter[4] Batch[156] Train-cross-entropy=1.831545
INFO:root:Iter[4] Batch[157] Train-cross-entropy=1.831899
INFO:root:Iter[4] Batch[158] Train-cross-entropy=1.830547
INFO:root:Iter

saving logfiles0  at resnet-101-fix0-imagenet-Mammalia-metric-log-0.txt


INFO:root:Iter[5] Batch[0] Train-cross-entropy=1.857003
INFO:root:Iter[5] Batch[1] Train-cross-entropy=1.796328
INFO:root:Iter[5] Batch[2] Train-cross-entropy=1.713598
INFO:root:Iter[5] Batch[3] Train-cross-entropy=1.740841
INFO:root:Iter[5] Batch[4] Train-cross-entropy=1.744696
INFO:root:Iter[5] Batch[5] Train-cross-entropy=1.717951
INFO:root:Iter[5] Batch[6] Train-cross-entropy=1.688123
INFO:root:Iter[5] Batch[7] Train-cross-entropy=1.726291
INFO:root:Iter[5] Batch[8] Train-cross-entropy=1.725112
INFO:root:Iter[5] Batch[9] Train-cross-entropy=1.735833
INFO:root:Iter[5] Batch[10] Train-cross-entropy=1.757434
INFO:root:Iter[5] Batch[11] Train-cross-entropy=1.781547
INFO:root:Iter[5] Batch[12] Train-cross-entropy=1.786777
INFO:root:Iter[5] Batch[13] Train-cross-entropy=1.789660
INFO:root:Iter[5] Batch[14] Train-cross-entropy=1.788082
INFO:root:Iter[5] Batch[15] Train-cross-entropy=1.787250
INFO:root:Iter[5] Batch[16] Train-cross-entropy=1.780044
INFO:root:Iter[5] Batch[17] Train-cross-e

INFO:root:Iter[5] Batch[142] Train-cross-entropy=1.779834
INFO:root:Iter[5] Batch[143] Train-cross-entropy=1.771814
INFO:root:Iter[5] Batch[144] Train-cross-entropy=1.769545
INFO:root:Iter[5] Batch[145] Train-cross-entropy=1.777286
INFO:root:Iter[5] Batch[146] Train-cross-entropy=1.772106
INFO:root:Iter[5] Batch[147] Train-cross-entropy=1.772166
INFO:root:Iter[5] Batch[148] Train-cross-entropy=1.771140
INFO:root:Iter[5] Batch[149] Train-cross-entropy=1.769729
INFO:root:Iter[5] Batch[150] Train-cross-entropy=1.767181
INFO:root:Iter[5] Batch[151] Train-cross-entropy=1.760387
INFO:root:Iter[5] Batch[152] Train-cross-entropy=1.763150
INFO:root:Iter[5] Batch[153] Train-cross-entropy=1.767117
INFO:root:Iter[5] Batch[154] Train-cross-entropy=1.769697
INFO:root:Iter[5] Batch[155] Train-cross-entropy=1.770218
INFO:root:Iter[5] Batch[156] Train-cross-entropy=1.773025
INFO:root:Iter[5] Batch[157] Train-cross-entropy=1.774243
INFO:root:Iter[5] Batch[158] Train-cross-entropy=1.773511
INFO:root:Iter

In [ ]:
checkpoint_path

In [ ]:


# In[ ]:
# 
train_score=[]
val_score=[]
epoch=[]
for i in range(0,5,120):
    
    sym, arg_params, aux_params =             mx.model.load_checkpoint(checkpoint_path, i)
        
    mod.set_params(arg_params, aux_params)
    
    #for gate in gatelist:
        #print mod.get_params()[0][gate].asnumpy(), gate
    res_train = mod.score(train_birds, mx.metric.Accuracy(),num_batch=44)
    res_val= mod.score(val_birds, mx.metric.Accuracy(),num_batch=44)
    epoch.append(i+1)
    for name, value in res_train:
        print 'Epoch[%d] Training-%s=%f' %(i+1, name, value)
        train_score.append(value)
    for name, value in res_val:
        print 'Epoch[%d] Validation-%s=%f' %(i+1, name, value)
        val_score.append(value)
        
#for gate in gatelist:
    #print mod.get_params()[0][gate].asnumpy(), gate


# In[ ]:

logfile_url= '%s-eval-metric-log-accuracy.txt' % (results_prefix+'eval/'+model_prefix)
print 'saving logfiles  at %s' % (logfile_url)
logfile = open(logfile_url, 'w')
for  epoch,train_metric, val_metric in zip(epoch,train_score,val_score):
    #logfile.write("%s\n" % item)


    logfile.write(str(epoch)+"\t"+str(train_metric)+"\t"+ str(val_metric)+"\n")
logfile.close()

